# Data Cleaning & Preprocessing - NBA Draft Prediction Project
In this notebook, we clean and prepare NCAA stats, NBA Combine measurements, and All-Star labels. The goal is to create a unified and structured dataset suitable for modeling.

# 02 Data Cleaning

*IDEAS*: Do difference between wingspan and height as variable

In [1]:
import pandas as pd
import numpy as np

## Renaming Players

## Cleaning NCAA Player Stats
We handle missing values, filter out incomplete seasons, and ensure consistent data types. This step ensures the statistical inputs for modeling are valid and comparable.

## Cleaning NBA Combine Data
Combine measurements are standardized and merged with NCAA player data. This involves cleaning strings, handling null values, and fixing duplicates.

In [2]:
combine = pd.read_csv('../data/raw/combine_data.csv')
ncaa = pd.read_csv('../data/raw/ncaa_data.csv')
combine = combine.query("`DRAFT YR` >= 2003") # Earliest NCAA yr is 2003

## Merging NCAA and Combine Data
We match NCAA players with their corresponding combine data using name and year. This step results in one master dataset for training.

In [3]:
merge_test = pd.merge(combine, ncaa, how= "outer",
                      left_on=['PLAYER', 'DRAFT YR'], right_on=['Player', 'Draft Yr'], 
                      indicator ='matched')
merge_test.query("matched=='left_only'")['PLAYER']

99               AJ Griffin
100              AJ Johnson
101               AJ Lawson
344               Adem Bona
479            Al Jefferson
                ...        
22197             Yves Pons
22208    Zaccharie Risacher
22251           Zach LaVine
22262      Zach Norvell Jr.
22369               Zhou Qi
Name: PLAYER, Length: 398, dtype: object

In [4]:
merge_test

,PLAYER,POS,BODY FAT %,HAND LENGTH (INCHES),HAND WIDTH (INCHES),HEIGHT W/O SHOES,HEIGHT W/ SHOES,STANDING REACH,WEIGHT (LBS),WINGSPAN,...,ORB,DRB,RPG,APG,SPG,BPG,TOV,PF,Draft Yr,matched
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.3,2.8,4.1,1.2,0.7,0.2,2.4,2.2,2014.0,right_only
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.9,2.6,3.6,0.8,0.9,0.2,2.0,2.0,2015.0,right_only
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.6,2.5,3.1,1.7,0.9,0.3,1.6,1.9,2024.0,right_only
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.8,3.8,5.6,0.5,0.8,0.8,1.9,2.8,2011.0,right_only
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.3,3.6,5.9,0.7,0.5,0.4,1.8,2.2,2012.0,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.1,8.2,10.3,3.2,0.8,0.8,3.0,2.7,2019.0,right_only
22394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.4,4.5,4.9,4.5,0.7,0.0,1.9,1.5,2021.0,right_only
22395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.4,5.2,5.6,4.3,0.8,0.0,2.3,1.6,2022.0,right_only
22396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.4,3.9,4.4,4.2,0.9,0.1,2.4,2.1,2023.0,right_only


#### *Comments:*

Looking at the results, some of these players were highly-touted but weren't high-scoring, which is how the data was sorted. So players like Cody Williams weren't in the top 1,000 in the NCAA in PPG

Different names: One of the last rows stands out, that being JT Toppin. The reason it didn't match to a dataset is because the NCAA dataset compiled from RealGM listed him as 'Jacob Toppin'. Therefore, it is important to pay special attention to players that participated in the combine that could have different nicknames

And finally, as anticipated, there were players that made it to the NBA without playing in the NCAA, whether that be through the G-League, high school, or overseas play

In [5]:
combine_nomatches = merge_test.query("matched=='left_only'")[['PLAYER', 'DRAFT YR']]
combine_nomatches['LAST NAME'] = combine_nomatches['PLAYER'].str.split(expand=True)[1]
ncaa_nomatches = merge_test.query("matched=='right_only'")[['Player', 'Draft Yr']]
ncaa_nomatches['Last Name'] = ncaa_nomatches['Player'].str.split(expand=True)[1]
ncaa_nomatches

,Player,Draft Yr,Last Name
0,A'Torri Shine,2014.0,Shine
1,A'Torri Shine,2015.0,Shine
2,A'lahn Sumler,2024.0,Sumler
3,A'uston Calhoun,2011.0,Calhoun
4,A'uston Calhoun,2012.0,Calhoun
...,...,...,...
22393,Zylan Cheatham,2019.0,Cheatham
22394,Zyon Pullin,2021.0,Pullin
22395,Zyon Pullin,2022.0,Pullin
22396,Zyon Pullin,2023.0,Pullin


In [6]:
mismatch = pd.merge(combine_nomatches, ncaa_nomatches, how="left",
                   left_on=['LAST NAME', 'DRAFT YR'], right_on=['Last Name', 'Draft Yr'],
                   indicator = 'matched')
mismatch.query("matched == 'both'")

,PLAYER,DRAFT YR,LAST NAME,Player,Draft Yr,Last Name,matched
0,AJ Griffin,2022.0,Griffin,Sam Griffin,2022.0,Griffin,both
1,AJ Griffin,2022.0,Griffin,Zion Griffin,2022.0,Griffin,both
2,AJ Johnson,2024.0,Johnson,Ben Johnson,2024.0,Johnson,both
3,AJ Johnson,2024.0,Johnson,Brandon Johnson,2024.0,Johnson,both
4,AJ Johnson,2024.0,Johnson,Butta Johnson,2024.0,Johnson,both
...,...,...,...,...,...,...,...
754,Willie Warren,2010.0,Warren,Chris Warren,2010.0,Warren,both
755,Willie Warren,2010.0,Warren,Jamarco Warren,2010.0,Warren,both
756,Willie Warren,2010.0,Warren,Storm Warren,2010.0,Warren,both
757,Xavier Tillman Sr.,2020.0,Tillman,Donnie Tillman,2020.0,Tillman,both


Since there appear to be duplicate last names that complicate matters, we will focus instead on the more distinguishable last names

In [7]:
mismatch = mismatch[~mismatch.duplicated(subset=['DRAFT YR', 'LAST NAME'], keep=False)]
mismatchnames = mismatch.query("matched == 'both'")
for index, row in mismatchnames.iterrows():
    print(f"'{row['Player']}': '{row['PLAYER']}',") # Dictionary format so relevant values can conform to combine naming

'Juan Wyatt': 'Avis Wyatt',
'Karim Coulibaly': 'Bilal Coulibaly',
'C.J. Elleby': 'CJ Elleby',
'C.J. Leslie': 'CJ Leslie',
'Cameron Christie': 'Cam Christie',
'Cam Reddish': 'Cameron Reddish',
'Bub Carrington': 'Carlton Carrington',
'Chuck Hayes': 'Charles Hayes',
'D.J. Steward': 'DJ Steward',
'D.J. Strawberry': 'DJ Strawberry',
'Lance Terry': 'Dalen Terry',
'J.R. Pinnock': 'Danilo Pinnock',
'David Duke': 'David Duke Jr.',
'DeAndre' Bembry': 'DeAndre Bembry',
'Jerome Jordan': 'DeAndre Jordan',
'Isaiah Walton': 'Derrick Walton Jr.',
'Thurman Zimmerman': 'Derrick Zimmerman',
'Devonte' Graham': 'Devonte Graham',
'Jamar Howard': 'Dwight Howard',
'Bam Adebayo': 'Edrice Adebayo',
'Brian Ibekwe': 'Ekene Ibekwe',
'Alpha Diallo': 'Hamidou Diallo',
'Brad Davison': 'JD Davison',
'J.J. Redick': 'JJ Redick',
'J.T. Toppin': 'JT Toppin',
'Elijah Hardy': 'Jaden Hardy',
'Jaylen Hoard': 'Jalen Hoard',
'Jimmer Fredette': 'James Fredette',
'Jaylen Sims': 'Jericho Sims',
'Austin Butler': 'John Butler Jr.',


In [8]:
ncaa['Player'] = ncaa['Player'].replace({
'C.J. Elleby': 'CJ Elleby',
'C.J. Leslie': 'CJ Leslie',
'Cameron Christie': 'Cam Christie',
'Cam Reddish': 'Cameron Reddish',
'Bub Carrington': 'Carlton Carrington',
'Chuck Hayes': 'Charles Hayes',
'D.J. Steward': 'DJ Steward',
'D.J. Strawberry': 'DJ Strawberry',
'J.R. Pinnock': 'Danilo Pinnock',
'David Duke': 'David Duke Jr.',
"DeAndre' Bembry": 'DeAndre Bembry',
"Devonte' Graham": 'Devonte Graham',
'Bam Adebayo': 'Edrice Adebayo',
'J.J. Redick': 'JJ Redick',
'J.T. Toppin': 'JT Toppin',
'Jaylen Hoard': 'Jalen Hoard',
'Jimmer Fredette': 'James Fredette',
'K.J. Simpson': 'KJ Simpson',
'Tiny Gallon': 'Keith Gallon',
'Kyle Lofton': 'Kenneth Lofton Jr.',
'Kenneth Kadji': 'Kenny Kadji',
'Matthew Hurt': 'Matt Hurt',
'Michael Frazier': 'Michael Frazier II',
'Mike Sweetney': 'Michael Sweetney',
'Mohammed Abukar': 'Mohamed Abukar',
'Moe Wagner': 'Moritz Wagner',
'Moe Wagner': 'Moritz Wagner',
"Nae'qwan Tomlin": "Nae'Qwan Tomlin",
'Bones Hyland': "Nah'Shon Hyland",
'Oshae Brissett': "O'Shae Brissett",
'P.J. Dozier': 'PJ Dozier',
'R.J. Barrett': 'RJ Barrett',
'Raiquan Gray': 'RaiQuan Gray',
'Raymond Spalding': 'Ray Spalding',
'Ricky Paulding': 'Rickey Paulding',
'Rob Dillingham': 'Robert Dillingham',
'Ron Harper': 'Ron Harper Jr.',
'Simi Shittu': 'Simisola Shittu',
'Svi Mykhailiuk': 'Sviatoslav Mykhailiuk',
'T.J. Cummings': 'TJ Cummings',
'T.J. Leaf': 'TJ Leaf',
'T.J. McConnell': 'TJ McConnell',
'Zach Norvell': 'Zach Norvell Jr.',})

## Reshaping Data

#### Combine Measurements

In [9]:
def get_measurements(col):
    cols = col.str.split("'", expand=True)
    cols[0] = cols[0].astype(float)
    cols[1] = cols[1].astype(float)
    return 12 * cols[0] + cols[1]

In [10]:
combine['HEIGHT W/O SHOES'] = get_measurements(combine['HEIGHT W/O SHOES'])
combine['WINGSPAN'] = get_measurements(combine['WINGSPAN'])

In [11]:
combine.head()

,PLAYER,POS,BODY FAT %,HAND LENGTH (INCHES),HAND WIDTH (INCHES),HEIGHT W/O SHOES,HEIGHT W/ SHOES,STANDING REACH,WEIGHT (LBS),WINGSPAN,DRAFT YR
225,Aloysius Anagonye,PF,10.50%,-,-,78.75,NaN,8' 10'',257.0,86.5,2003
226,Rick Anderson,SF-PF,9.90%,-,-,80.25,NaN,8' 10.5'',216.0,83.0,2003
227,Carmelo Anthony,SF,8.00%,-,-,78.25,NaN,8' 9.5'',233.0,84.0,2003
228,Jerome Beasley,SF-PF,9.90%,-,-,81.00,NaN,8' 11'',237.0,84.5,2003
229,Troy Bell,PG,5.30%,-,-,72.25,NaN,8' 1.5'',178.0,77.0,2003


In [12]:
combine_clean = combine[['PLAYER', 'DRAFT YR', 'HEIGHT W/O SHOES', 'WEIGHT (LBS)', 'WINGSPAN']]
combine_clean = combine_clean.rename({'PLAYER': 'Player', 
                                      'DRAFT YR': 'Draft Year', 
                                      'HEIGHT W/O SHOES': 'Height (in)', 
                                      'WEIGHT (LBS)': 'Weight (lbs)',
                                      'WINGSPAN': 'Wingspan (in)'}, axis=1)

In [13]:
combine_clean['Wingspan diff'] = combine_clean['Wingspan (in)'] - combine_clean['Height (in)']

In [14]:
combine_clean.to_csv('../data/processed/combine_data.csv', index=False)

In [15]:
ncaa.to_csv('../data/processed/ncaa_data.csv', index=False)

## Merging

In [16]:
all_data = pd.merge(combine_clean, ncaa, how="inner",
                   left_on=['Player', 'Draft Year'], right_on=['Player', 'Draft Yr'])
all_data = all_data.drop(['Draft Yr', '#'], axis=1) # Removing unnecessary Draft Yr and rank columns

## Integrating All-Star Labels
We merge the cleaned dataset with All-Star labels. A binary column `all_star` is created for supervised learning.

In [19]:
allstars = pd.read_csv('../data/raw/allstars.csv')
allstars['All-Star'] = 'Yes'
all_data = pd.merge(all_data, allstars, how="left",
                   on='Player')
all_data.fillna({'All-Star': 'No'}, inplace=True)
all_data

,Player,Draft Year,Height (in),Weight (lbs),Wingspan (in),Wingspan diff,Team,GP,MPG,PPG,...,ORB,DRB,RPG,APG,SPG,BPG,TOV,PF,All-Star_x,All-Star_y
0,Rick Anderson,2003,80.25,216.0,83.00,2.75,ARZ,31,25.4,10.7,...,2.6,4.1,6.7,1.5,0.8,0.8,1.3,2.7,No,NaN
1,Carmelo Anthony,2003,78.25,233.0,84.00,5.75,SYRA,35,36.4,22.2,...,2.9,6.9,9.7,2.2,1.5,0.9,2.2,2.2,Yes,Yes
2,Troy Bell,2003,72.25,178.0,77.00,4.75,BOS,31,38.6,25.2,...,1.5,3.0,4.6,3.7,2.3,0.2,2.5,2.1,No,NaN
3,LaVell Blanchard,2003,78.00,205.0,78.00,0.00,MICH,30,32.0,16.2,...,2.1,5.0,7.1,1.0,0.6,0.3,2.0,2.6,No,NaN
4,Keith Bogans,2003,76.25,213.0,81.25,5.00,UK,36,29.7,15.7,...,1.1,2.7,3.8,2.7,1.2,0.1,2.2,1.7,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137,Jaylon Tyson,2024,77.50,218.2,80.00,2.50,Cal,31,34.3,19.6,...,1.6,5.1,6.8,3.5,1.2,0.5,3.1,2.1,No,NaN
1138,Ja'Kobe Walter,2024,76.25,197.6,82.00,5.75,BAYL,35,32.3,14.5,...,1.3,3.1,4.4,1.4,1.1,0.2,1.3,2.1,No,NaN
1139,Kel'el Ware,2024,83.75,230.0,88.50,4.75,IU,30,32.2,15.9,...,2.1,7.8,9.9,1.5,0.6,1.9,1.6,2.0,No,NaN
1140,Jamir Watkins,2024,77.00,211.6,82.75,5.75,FLAST,33,28.2,15.6,...,1.5,4.5,6.0,2.8,1.9,0.8,2.5,2.8,No,NaN


In [18]:
all_data.to_csv('../data/processed/merged_data.csv', index=False)